<a href="https://colab.research.google.com/github/amal572/Amal/blob/master/ontoloi_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install owlready2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.4 MB 1.4 MB/s 
  Created wheel for owlready2: filename=Owlready2-0.38-cp37-cp37m-linux_x86_64.whl size=22127093 sha256=0b1f40c224719bbcddf829827b07add1598ce7344866501b5d43edf2ffa42d23
  Stored in directory: /root/.cache/pip/wheels/ee/48/58/3ea20d69efcb97a26048fbfcce9332412077df80d4ae52365b
Successfully built owlready2


In [2]:
import spacy
from spacy.lang.en import English
import networkx as nx
import matplotlib.pyplot as plt
from owlready2 import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

In [3]:
per = get_ontology("perceptron.owl").load()

In [4]:
per.base_iri

'http://www.w3.org/ns/perceptron.owl#'

In [5]:
list(per.classes())

[perceptron.Add,
 perceptron.Algorithm,
 perceptron.Dataset,
 perceptron.Derviation,
 perceptron.Div,
 perceptron.EarlyStop,
 perceptron.EvaluationMetric,
 perceptron.Feature,
 perceptron.FeedForward,
 perceptron.Formula,
 perceptron.Function,
 perceptron.HyperParameter,
 perceptron.Initializer,
 perceptron.Layer,
 perceptron.Limitation,
 perceptron.Logarithm,
 perceptron.Model,
 perceptron.Mult,
 perceptron.Operator,
 perceptron.OptimizationAlgorithm,
 perceptron.Optimizer,
 perceptron.Output,
 perceptron.Parameter,
 perceptron.Power,
 perceptron.Root,
 perceptron.Sigma,
 perceptron.Sub,
 perceptron.Task,
 perceptron.Validator,
 perceptron.Value,
 perceptron.input]

In [6]:
list(per.properties())

[perceptron.adjustLeraningRate,
 perceptron.description,
 perceptron.formula,
 perceptron.isSOTA,
 perceptron.isSupervised,
 perceptron.label,
 perceptron.lang,
 perceptron.name,
 perceptron.sample,
 perceptron.synonym,
 perceptron.type,
 perceptron.actOn,
 perceptron.adjustParameter,
 perceptron.canExtract,
 perceptron.canGetData,
 perceptron.evaluatedBy,
 perceptron.excutes,
 perceptron.fit,
 perceptron.hasExample,
 perceptron.hasFormula,
 perceptron.hasHyperParameter,
 perceptron.hasInput,
 perceptron.hasLayer,
 perceptron.hasModel,
 perceptron.hasNext,
 perceptron.isPartOf,
 perceptron.learns,
 perceptron.produce,
 perceptron.solvedBy,
 perceptron.sufferFrom,
 perceptron.train,
 perceptron.useAlgoritm,
 perceptron.useFunction,
 perceptron.useOptimizer,
 perceptron.uses]

In [7]:
list(default_world.sparql("""SELECT ?domain ?property ?range
WHERE{  
       ?property rdf:type owl:ObjectProperty .
       ?property rdfs:range ?range .
       ?property rdfs:domain ?domain
     }"""))

[[perceptron.Limitation, perceptron.actOn, perceptron.Parameter],
 [perceptron.Initializer, perceptron.adjustParameter, perceptron.Parameter],
 [perceptron.Optimizer, perceptron.adjustParameter, perceptron.Parameter],
 [perceptron.input, perceptron.canExtract, perceptron.Feature],
 [perceptron.input, perceptron.canGetData, perceptron.Dataset],
 [perceptron.Output, perceptron.evaluatedBy, perceptron.EvaluationMetric],
 [perceptron.Algorithm, perceptron.excutes, perceptron.Operator],
 [perceptron.Algorithm, perceptron.hasExample, perceptron.Task],
 [perceptron.Algorithm,
  perceptron.hasHyperParameter,
  perceptron.HyperParameter],
 [perceptron.EarlyStop,
  perceptron.hasHyperParameter,
  perceptron.HyperParameter],
 [perceptron.FeedForward,
  perceptron.hasHyperParameter,
  perceptron.HyperParameter],
 [perceptron.Initializer,
  perceptron.hasHyperParameter,
  perceptron.HyperParameter],
 [perceptron.Optimizer,
  perceptron.hasHyperParameter,
  perceptron.HyperParameter],
 [perceptron.V

In [8]:
list(default_world.sparql("""SELECT  ?x
WHERE { 
  ?x rdf:type owl:Class 
}"""))

[[perceptron.Add],
 [perceptron.Algorithm],
 [perceptron.Dataset],
 [perceptron.Derviation],
 [perceptron.Div],
 [perceptron.EarlyStop],
 [perceptron.EvaluationMetric],
 [perceptron.Feature],
 [perceptron.FeedForward],
 [perceptron.Formula],
 [perceptron.Function],
 [perceptron.HyperParameter],
 [perceptron.Initializer],
 [perceptron.Layer],
 [perceptron.Limitation],
 [perceptron.Logarithm],
 [perceptron.Model],
 [perceptron.Mult],
 [perceptron.Operator],
 [perceptron.OptimizationAlgorithm],
 [perceptron.Optimizer],
 [perceptron.Output],
 [perceptron.Parameter],
 [perceptron.Power],
 [perceptron.Root],
 [perceptron.Sigma],
 [perceptron.Sub],
 [perceptron.Task],
 [perceptron.Validator],
 [perceptron.Value],
 [perceptron.input]]

In [19]:
def getSentences(text):
    en_nlp = spacy.load('en_core_web_sm')
    document = en_nlp(text)
    return [str(sent).strip() for sent in document.sents]

def printToken(token):
    print(token.text, "->", token.dep_)

def appendChunk(original, chunk):
    return original + ' ' + chunk

def isRelationCandidate(token):
    deps = ["ROOT", "adj", "attr", "agent", "amod"]
    return any(subs in token.dep_ for subs in deps)

def isConstructionCandidate(token):
    deps = ["compound", "prep", "conj", "mod"]
    return any(subs in token.dep_ for subs in deps)

def processSubjectObjectPairs(tokens):
    subject = ''
    objectt = ''
    relation = ''
    subjectConstruction = ''
    objectConstruction = ''
    for token in tokens:
        printToken(token)
        if "punct" in token.dep_:
            continue
        if isRelationCandidate(token):
            relation = appendChunk(relation, token.lemma_)
        if isConstructionCandidate(token):
            if subjectConstruction:
                subjectConstruction = appendChunk(subjectConstruction, token.text)
            if objectConstruction:
                objectConstruction = appendChunk(objectConstruction, token.text)
        if "subj" in token.dep_:
            subject = appendChunk(subject, token.text)
            subject = appendChunk(subjectConstruction, subject)
            subjectConstruction = ''
        if "obj" in token.dep_:
            objectt = appendChunk(objectt, token.text)
            objectt = appendChunk(objectConstruction, objectt)
            objectConstruction = ''
    
    print ('subject', subject.strip())
    print ('relation',relation.strip())
    print ("object", objectt.strip())
    return (subject.strip(), relation.strip(), objectt.strip())

def processSentence(sentence):
    tokens = nlp_model(sentence)
    return processSubjectObjectPairs(tokens)

def printGraph(triples):
    G = nx.Graph()
    for triple in triples:
        G.add_node(triple[0])
        G.add_node(triple[1])
        G.add_node(triple[2])
        G.add_edge(triple[0], triple[1])
        G.add_edge(triple[1], triple[2])

    pos = nx.spring_layout(G)
    plt.figure()
    nx.draw(G, pos, edge_color='black', width=1, linewidths=1,
            node_size=500, node_color='seagreen', alpha=0.9,
            labels={node: node for node in G.nodes()})
    plt.axis('off')
    plt.show()



In [27]:
text = "what is the HyperParameter of MLP"

sentences = getSentences(text)
nlp_model = spacy.load('en_core_web_sm')

triples = []
print (text)
for sentence in sentences:
    triples.append(processSentence(sentence))

#printGraph(triples)


what is HyperParameter of MLP
what -> attr
is -> ROOT
HyperParameter -> nsubj
of -> prep
MLP -> pobj
subject HyperParameter
relation what be
object MLP


In [28]:
x = " "
y = " "
for i in triples:
    x = i[1]
    y = i[2]

In [29]:
s = []
cc = list(per.properties()) 
for i in cc:
    s.append(i.name)

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

string = x
string_list = s

tfidf_vectorizer = TfidfVectorizer(analyzer="char")

sparse_matrix = tfidf_vectorizer.fit_transform([string]+string_list)
cosine = cosine_similarity(sparse_matrix[0,:],sparse_matrix[1:,:])

df = pd.DataFrame({'cosine':cosine[0],'strings':string_list}).sort_values('cosine',ascending=False)
x = df['strings'].iloc[0]
print(x)

evaluatedBy


In [34]:
list(default_world.sparql(
"""
   PREFIX custom:  <http://www.w3.org/ns/perceptron.owl#>

   SELECT ?subject ?object

   WHERE {?subject custom:hasHyperParameter ?object   
        }

"""))

[[perceptron.EarlyStop_validation_fraction,
  perceptron.EarlyStop_validation_fraction],
 [perceptron.MLP, perceptron.LearningRate],
 [perceptron.MLP, perceptron.MaxIter],
 [perceptron.MLP, perceptron.RegularizationAlpha],
 [perceptron.MLP, perceptron.RegularizationType],
 [perceptron.MLP, perceptron.Tolerence],
 [perceptron.MLPEarlyStop, perceptron.EarlyStop_validation_fraction],
 [perceptron.MLPRandomInitializer, perceptron.RandomState],
 [perceptron.Perceptron, perceptron.LearningRate],
 [perceptron.Perceptron, perceptron.MaxIter],
 [perceptron.Perceptron, perceptron.Tolerence],
 [perceptron.PerceptronRandomInitializer, perceptron.RandomState]]

In [35]:
relation = 'hasHyperParameter'
objec ='Perceptron'
list(default_world.sparql(
"""
    PREFIX custom:  <http://www.w3.org/ns/perceptron.owl#>
    SELECT ?sender WHERE {
     custom:%s custom:%s ?sender .
    }

"""%(objec,relation)))

[[perceptron.LearningRate], [perceptron.MaxIter], [perceptron.Tolerence]]

In [36]:
relation = 'hasHyperParameter'
objec ='Perceptron'
list(default_world.sparql(
"""
    PREFIX custom:  <http://www.w3.org/ns/perceptron.owl#>
    SELECT ?sender ?relation WHERE {
     custom:%s ?relation ?sender .
    }

"""%(objec)))

[[owl.NamedIndividual, 6],
 [perceptron.Algorithm, 6],
 [perceptron.PerceptronRandomInitializer, perceptron.excutes],
 [perceptron.BinaryClassification, perceptron.hasExample],
 [perceptron.LearningRate, perceptron.hasHyperParameter],
 [perceptron.MaxIter, perceptron.hasHyperParameter],
 [perceptron.Tolerence, perceptron.hasHyperParameter],
 [perceptron.SingleLayer, perceptron.hasLayer],
 [perceptron.Bias, perceptron.learns],
 [weights-and-biases.Weight, perceptron.learns],
 [perceptron.NonLinearData, perceptron.sufferFrom],
 [perceptron.HingeLoss, perceptron.useFunction],
 [perceptron.LeakyReLUActivation, perceptron.useFunction],
 [perceptron.LinearActivation, perceptron.useFunction],
 [perceptron.ReLUActivation, perceptron.useFunction],
 [perceptron.StepActivation, perceptron.useFunction],
 [.SGD, perceptron.useOptimizer],
 [.SGDwithMomentum, perceptron.useOptimizer],
 [True, perceptron.isSupervised],
 ['perceptron', perceptron.name],
 ['activation', perceptron.synonym],
 ['neuron', 

In [31]:
relation = x
objec = y
xx =list(default_world.sparql(
"""
    PREFIX custom:  <http://www.w3.org/ns/perceptron.owl#>
    SELECT ?sender WHERE {
     custom:%s custom:%s ?sender .
    }

"""%(objec,relation)))


In [33]:
s = []
ss =' and '
for i in xx:
    for z in i:
        s.append(z.name)

mylist = list(dict.fromkeys(s))
ss = ss.join(mylist)
su = objec
re = relation
q = "the " + su + " " + re + " its "+ ss
print(q)

the MLP evaluatedBy its 
